# Step-B FineTune 训练

### 1.加载VGG预训练参数，加载FC层参数

### 2.训练CONV层和FC层

### 2.保存CONV层和FC层参数

In [3]:
import os
import random
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from data_preprocess import *
from image_preprocess import *
from bcnn_model import *

In [4]:
weights_file = "vgg16_weights.npz"
weights_file_last_layer = "bcnn_last_weights.npz"
weights_file_conv = "bcnn_conv_weights.npz"

batch_size = 24
lr = 0.001
finetune_step = -1

image_size = 224
n_classes = 12

epoch = 200

In [5]:
tf.reset_default_graph()
sess = tf.Session()
# build input
imgs = tf.placeholder(tf.float32, shape = [None, image_size, image_size, 3], name = "input_x")
targets = tf.placeholder(tf.int32, shape = [None, n_classes], name = "input_y")
keep_prob = tf.placeholder(tf.float32, name = "keep_prob")

# build cal graph
bilinear_cnn = vgg_bilinear_model(imgs, 
                                  keep_prob, 
                                  weights_file = weights_file, 
                                  weights_file_last = weights_file_last_layer, 
                                  sess = sess, 
                                  finetune = True)

# build cal node parameter
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = bilinear_cnn.fc3l)
cost = tf.reduce_mean(cross_entropy, name = "cost")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

predicted = tf.nn.softmax(bilinear_cnn.fc3l, name = "predicted")
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name = "accuracy")

# start train
sess.run(tf.global_variables_initializer())
bilinear_cnn.load_vgg_weights(sess)
bilinear_cnn.load_last_layer_weights(sess)

for v in tf.trainable_variables():
    print("Trainable variables", v)
print('Starting training')


###################################
train_img_batch, train_label_batch = get_batchs(tfrecord_filename = "train.tfrecords", 
                                                    image_size = image_size, classes = n_classes,
                                                    batch_size = batch_size, min_after_dequeue = 500,
                                                   use_data_enhancement = True)
test_img_batch, test_label_batch = get_batchs(tfrecord_filename = "test.tfrecords", 
                                                  image_size = image_size, classes = n_classes,
                                                  batch_size = batch_size, min_after_dequeue = 500,
                                                     use_data_enhancement = False)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(int((4512 / batch_size) * epoch)):
#for i in range(2):
    # get data
    train_imgs, train_labels= sess.run([train_img_batch, train_label_batch])
    test_imgs, test_labels= sess.run([test_img_batch, test_label_batch])
    
#    train_imgs = image_random_enhancement(train_imgs)

    # train single loop
    _, train_loss = sess.run([optimizer,cost], feed_dict={imgs: train_imgs, targets: train_labels, keep_prob: 0.5})
    #train_loss = sess.run(cost, feed_dict={imgs: train_imgs, targets: train_labels, keep_prob: 1.})
    print("train loss: %10f"%(train_loss))
            
    # print test stat
    if (i % 50 == 0):
        print("########### epoch %d ###########"%((i / (4512 / batch_size)) + 1))
        print("########### loop %d ###########"%(i))
        test_loss, test_acc = sess.run([cost, accuracy], feed_dict={imgs: test_imgs, targets: test_labels, keep_prob: 1.})
        print("test loss: %10f   test accuracy: %10f"%(test_loss, test_acc))
        print("########### loop %d ###########"%(i))

# close data queue
coord.request_stop()
coord.join(threads)

# save last layer parameters
last_layer_weights = []

for var in bilinear_cnn.last_layer_parameters:
    last_layer_weights.append(sess.run(var))

np.savez(weights_file_last_layer, last_layer = last_layer_weights)

# save conv layer parameters
conv_layer_weights = []

for var in bilinear_cnn.parameters:
    conv_layer_weights.append(sess.run(var))

np.savez(weights_file_conv, conv_layer = conv_layer_weights)

print('done')
 
# end
sess.close()

Adding Data Augmentation


D:\Users\XX\Anaconda2\envs\gpu-env-tf\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Shape of z (?, 262144)
Adding weights to conv1_1/b:0
Adding weights to conv1_1/W:0
Adding weights to conv1_2/b:0
Adding weights to conv1_2/W:0
Adding weights to conv2_1/b:0
Adding weights to conv2_1/W:0
Adding weights to conv2_2/b:0
Adding weights to conv2_2/W:0
Adding weights to conv3_1/b:0
Adding weights to conv3_1/W:0
Adding weights to conv3_2/b:0
Adding weights to conv3_2/W:0
Adding weights to conv3_3/b:0
Adding weights to conv3_3/W:0
Adding weights to conv4_1/b:0
Adding weights to conv4_1/W:0
Adding weights to conv4_2/b:0
Adding weights to conv4_2/W:0
Adding weights to conv4_3/b:0
Adding weights to conv4_3/W:0
Adding weights to conv5_1/b:0
Adding weights to conv5_1/W:0
Adding weights to conv5_2/b:0
Adding weights to conv5_2/W:0
Adding weights to conv5_3/b:0
Adding weights to conv5_3/W:0
last_layer [[ 9.5026232e-02  7.2930790e-02 -3.6813654e-02 ... -3.4357134e-02
  -8.7767147e-02  5.0819542e-02]
 [ 1.3950452e-03 -1.4508091e-03 -3.7143724e-03 ...  4.9801045e-03
  -6.6605175e-04 -8.1

train loss:   1.305809
train loss:   1.304635
train loss:   1.154440
train loss:   0.971507
train loss:   1.066664
train loss:   1.235158
train loss:   1.633465
train loss:   1.286285
train loss:   1.748744
train loss:   1.336265
train loss:   1.263767
train loss:   1.392297
train loss:   1.183750
train loss:   1.431417
train loss:   1.565588
train loss:   1.072053
train loss:   1.339392
train loss:   1.523391
train loss:   1.172826
train loss:   1.765412
train loss:   1.287030
train loss:   1.120016
train loss:   1.016456
train loss:   1.376207
train loss:   1.350168
train loss:   0.815626
train loss:   1.181934
train loss:   1.210267
train loss:   1.253849
train loss:   1.107709
train loss:   1.355977
train loss:   1.191433
train loss:   1.270161
train loss:   1.547381
train loss:   1.595149
train loss:   1.358863
########### epoch 2 ###########
########### loop 200 ###########
test loss:   0.573254   test accuracy:   1.000000
########### loop 200 ###########
train loss:   1.174574
t

train loss:   1.488551
train loss:   1.029090
train loss:   1.191346
train loss:   0.893130
train loss:   1.129353
train loss:   1.196782
train loss:   1.102425
train loss:   1.467123
train loss:   1.106547
train loss:   1.368871
train loss:   1.342979
train loss:   1.243079
train loss:   1.436270
train loss:   0.938568
train loss:   1.122082
train loss:   1.026681
train loss:   0.598622
train loss:   1.023836
########### epoch 3 ###########
########### loop 500 ###########
test loss:   0.479470   test accuracy:   0.958333
########### loop 500 ###########
train loss:   1.238457
train loss:   1.310407
train loss:   1.219466
train loss:   0.992456
train loss:   0.942778
train loss:   1.274207
train loss:   1.176272
train loss:   1.642425
train loss:   1.233101
train loss:   1.443221
train loss:   1.088242
train loss:   1.049178
train loss:   1.347713
train loss:   1.145346
train loss:   1.320932
train loss:   1.292358
train loss:   1.116151
train loss:   1.119900
train loss:   0.956854
t

...

train loss:   0.790713
train loss:   1.168181
train loss:   1.255845
########### epoch 193 ###########
########### loop 36100 ###########
test loss:   0.261303   test accuracy:   0.916667
########### loop 36100 ###########
train loss:   1.296659
train loss:   1.062654
train loss:   0.957599
train loss:   0.596737
train loss:   0.897855
train loss:   1.006039
train loss:   0.902048
train loss:   0.736212
train loss:   1.103368
train loss:   0.931630
train loss:   1.239196
train loss:   1.037085
train loss:   0.729824
train loss:   1.386737
train loss:   1.140017
train loss:   0.747387
train loss:   0.817582
train loss:   0.886132
train loss:   1.068633
train loss:   0.913519
train loss:   0.834370
train loss:   0.988068
train loss:   0.859194
train loss:   0.946131
train loss:   1.113912
train loss:   0.928537
train loss:   1.352097
train loss:   1.030365
train loss:   0.472116
train loss:   0.884145
train loss:   0.708774
train loss:   0.943090
train loss:   0.984639
train loss:   0.90

train loss:   0.928128
train loss:   1.079687
train loss:   0.893673
train loss:   0.999328
train loss:   0.683613
train loss:   0.660005
train loss:   0.914046
train loss:   1.015059
train loss:   1.058547
train loss:   1.198835
train loss:   0.761686
train loss:   0.582304
train loss:   1.455875
train loss:   0.972362
train loss:   1.055801
train loss:   0.671349
train loss:   0.767556
train loss:   0.939021
train loss:   0.847105
train loss:   0.905698
train loss:   0.979052
train loss:   0.914037
train loss:   0.636118
train loss:   1.025372
train loss:   1.124743
train loss:   0.793755
train loss:   0.853570
train loss:   1.176149
train loss:   0.771850
train loss:   0.959582
train loss:   1.107026
train loss:   0.862209
train loss:   0.886108
train loss:   1.274233
train loss:   1.009994
train loss:   0.927958
train loss:   1.087782
train loss:   0.997223
train loss:   0.778011
train loss:   0.946343
train loss:   1.050429
train loss:   0.979993
train loss:   1.055156
###########

train loss:   0.829568
train loss:   0.882400
train loss:   0.913965
train loss:   1.047444
train loss:   0.971555
train loss:   1.125588
train loss:   1.103075
train loss:   0.843240
train loss:   0.901331
train loss:   1.134526
train loss:   1.232153
train loss:   0.995118
train loss:   1.039089
train loss:   1.151705
train loss:   1.154601
train loss:   0.513249
train loss:   1.062318
train loss:   0.856046
train loss:   0.991575
train loss:   0.724980
train loss:   1.073919
train loss:   0.783942
train loss:   0.834556
train loss:   1.098016
train loss:   0.880003
train loss:   1.193455
########### epoch 196 ###########
########### loop 36750 ###########
test loss:   0.373984   test accuracy:   0.875000
########### loop 36750 ###########
train loss:   0.986334
train loss:   0.708840
train loss:   0.812642
train loss:   0.915163
train loss:   0.686805
train loss:   1.119224
train loss:   1.072977
train loss:   0.980434
train loss:   0.606378
train loss:   1.106594
train loss:   0.87

train loss:   1.396979
train loss:   0.859125
train loss:   1.056163
train loss:   0.925622
train loss:   0.767188
train loss:   1.057406
train loss:   0.615390
train loss:   1.351120
train loss:   0.788239
########### epoch 198 ###########
########### loop 37050 ###########
test loss:   0.155537   test accuracy:   1.000000
########### loop 37050 ###########
train loss:   1.112106
train loss:   0.861732
train loss:   1.047012
train loss:   1.194531
train loss:   1.286505
train loss:   1.001562
train loss:   0.847456
train loss:   1.070141
train loss:   0.902062
train loss:   0.699558
train loss:   0.919065
train loss:   0.909393
train loss:   1.368153
train loss:   0.939400
train loss:   0.660377
train loss:   0.762833
train loss:   1.031051
train loss:   1.109593
train loss:   0.864013
train loss:   1.222270
train loss:   0.903784
train loss:   1.409072
train loss:   0.407273
train loss:   0.966489
train loss:   0.697725
train loss:   1.062691
train loss:   0.967436
train loss:   0.77

train loss:   0.960229
train loss:   1.169029
train loss:   0.735311
train loss:   0.775417
train loss:   0.955036
train loss:   0.893158
train loss:   0.916462
train loss:   1.045383
train loss:   0.876334
train loss:   0.802858
train loss:   0.944323
train loss:   1.180802
train loss:   0.837481
train loss:   1.216418
train loss:   0.876014
train loss:   0.889124
train loss:   0.993264
train loss:   0.459879
train loss:   0.794501
train loss:   0.716717
train loss:   0.810923
train loss:   1.009451
train loss:   0.860826
train loss:   0.586444
train loss:   0.941597
train loss:   1.101914
train loss:   0.855387
train loss:   0.595361
train loss:   1.130717
train loss:   0.984826
train loss:   1.068027
train loss:   1.068730
train loss:   0.761801
train loss:   0.737065
train loss:   0.806993
train loss:   1.068618
train loss:   1.015931
train loss:   1.241793
train loss:   0.935030
train loss:   1.234833
train loss:   0.880537
train loss:   1.145246
train loss:   0.613566
train loss: